# 第三章 迭代优化

当使用 LLM 构建应用程序时，实践层面上很难*第一次尝试*就成功获得适合最终应用的 Prompt。但这并不重要，只要您有一个好的迭代过程来不断改进您的 Prompt，那么您就能够得到一个适合任务的 Prompt。虽然相比训练机器学习模型，在  Prompt 方面一次成功的几率可能会高一些，但正如上所说， Prompt 是否一次完善并不重要。最重要的是**层层迭代**为您的应用程序找到有效  Prompt 的过程。

因此在本章中，我们将以产品说明书中生成营销文案为例，来展示一些流程框架，并提示您思考如何层层迭代地分析和完善您的 Prompt。

在吴恩达（Andrew Ng，原教程作者）的机器学习课程中展示过一张图表，说明了机器学习开发的流程。通常是先有一个想法，然后再用以下流程实现：编写代码，获取数据，训练模型，获得实验结果。然后您可以查看结果，分析误差与错误，找出适用领域，甚至可以更改您对具体问题的具体思路或解决方法。此后再次更改实现，并运行另一个实验等，反复迭代，最终获得有效的机器学习模型。在编写基于 LLM 的应用程序的 Prompt 时，流程可能非常相似。您产生了关于要完成的任务的想法后，可以尝试编写第一个 Prompt ，注意要满足上一章说过的两个原则：**清晰明确，并且给系统足够的时间思考**。然后您可以运行并查看结果。如果第一次效果不好，那么迭代的过程就是找出为什么指令不够清晰或为什么没有给算法足够的时间思考，以便改进想法、改进  Prompt 等等，循环多次，直到找到适合您的应用程序的 Prompt。

很难有适用于世间万物的所谓“最佳  Prompt ”，更好的方法是找到有效的迭代过程，以便您可以快速地找到一个适合您的应用程序的  Prompt 。


<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一环境配置" data-toc-modified-id="一、环境配置">一、环境配置</a></span></li>
        <li>
            <span><a href="#二任务从产品说明书生成一份营销产品描述" data-toc-modified-id="二、任务——从产品说明书生成一份营销产品描述">二、任务——从产品说明书生成一份营销产品描述</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-问题一生成文本太长" data-toc-modified-id="2.1 问题一：生成文本太长">2.1 问题一：生成文本太长</a></span></li>
                <li><span><a href="#22-问题二抓错文本细节" data-toc-modified-id="2.2 问题二：抓错文本细节">2.2 问题二：抓错文本细节</a></span></li>
                <li><span><a href="#23-问题三添加表格描述" data-toc-modified-id="2.3 问题三：添加表格描述">2.3 问题三：添加表格描述</a></span></li>
            </ul>
        </li>
    </ul>
</div>

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [1]:
import openai
# 导入第三方库

import os
openai.api_key = os.environ.get("OPENAI_API_KEY")
# 设置 API_KEY, 请替换成您自己的 API_KEY
deployment_id = None
API_TYPE=os.getenv("OPENAI_API_TYPE", None)
if not API_TYPE is None:
    openai.api_type=API_TYPE
    openai.api_base=os.environ["OPENAI_API_BASE"]
    openai.api_version=os.environ["OPENAI_API_VERSION"]
    deployment_id=os.getenv("DEPLOYMENT_ID", "gpt-35-turbo")

In [2]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=deployment_id,
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


## 二、任务——从产品说明书生成一份营销产品描述

给定一份椅子的资料页。描述说它属于*中世纪灵感*系列，产自意大利，并介绍了材料、构造、尺寸、可选配件等参数。假设您想要使用这份说明书帮助营销团队为电商平台撰写营销描述稿：

In [3]:
# 示例：产品说明书
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
#   Prompt ：基于说明书生成营销描述
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. This chair is part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, all designed with a timeless mid-century aesthetic.

One of the standout features of this chair is the variety of customization options available. You can choose from several shell colors and base finishes to perfectly match your existing decor. The chair is available with either plastic back and front upholstery or full upholstery in a range of 10 fabric and 6 leather options, allowing you to create a look that is uniquely yours.

The chair is also available with or without armrests, giving you the flexibility to choose the option that best suits your needs. The base finish options include stainless steel, matte black, gloss white, or chrome, ensuring that you can find the perfect match for your space.

In terms of construction, this chair is

In [5]:
# 示例：产品说明书
fact_sheet_chair = """
概述

    美丽的中世纪风格办公家具系列的一部分，包括文件柜、办公桌、书柜、会议桌等。
    多种外壳颜色和底座涂层可选。
    可选塑料前后靠背装饰（SWC-100）或10种面料和6种皮革的全面装饰（SWC-110）。
    底座涂层选项为：不锈钢、哑光黑色、光泽白色或铬。
    椅子可带或不带扶手。
    适用于家庭或商业场所。
    符合合同使用资格。

结构

    五个轮子的塑料涂层铝底座。
    气动椅子调节，方便升降。

尺寸

    宽度53厘米|20.87英寸
    深度51厘米|20.08英寸
    高度80厘米|31.50英寸
    座椅高度44厘米|17.32英寸
    座椅深度41厘米|16.14英寸

选项

    软地板或硬地板滚轮选项。
    两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。
    无扶手或8个位置PU扶手。

材料
外壳底座滑动件

    改性尼龙PA6/PA66涂层的铸铝。
    外壳厚度：10毫米。
    座椅
    HD36泡沫

原产国

    意大利
"""

In [6]:
#   Prompt ：基于说明书创建营销描述
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的营销描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

技术说明: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


产品描述：

我们自豪地推出美丽的中世纪风格办公家具系列，其中包括文件柜、办公桌、书柜和会议桌等多种产品。这个系列提供了多种外壳颜色和底座涂层选择，以满足不同客户的个性化需求。您可以选择塑料前后靠背装饰（SWC-100）或全面装饰（SWC-110），全面装饰提供了10种面料和6种皮革供您选择。底座涂层选项包括不锈钢、哑光黑色、光泽白色或铬。椅子可以选择带扶手或不带扶手，以适应不同的使用场景，无论是家庭还是商业场所。

我们的产品不仅外观精美，而且结构稳固。底座采用五个轮子的塑料涂层铝底座，方便移动和固定。椅子采用气动调节设计，可以轻松调节座椅的高度，以满足不同用户的需求。

产品的尺寸为宽度53厘米、深度51厘米、高度80厘米，座椅高度为44厘米，座椅深度为41厘米。这个尺寸设计既舒适又符合人体工学原理，让您在使用时感到舒适自在。

我们还提供了一些选项供您选择。您可以根据使用环境选择软地板或硬地板滚轮。座椅泡沫密度可选中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。此外，您还可以选择无扶手或8个位置PU扶手，以满足不同用户的需求。

我们的产品采用高质量的材料制造而成。外壳底座滑动件采用改性尼龙PA6/PA66涂层的铸铝，具有出色的耐用性和稳定性。外壳厚度为10毫米，确保产品的结构牢固。座椅采用HD36泡沫，提供舒适的坐感。

我们的产品是意大利制造，以其精湛的工艺和高品质而闻名。我们致力于为客户提供优质的办公家具，让您的办公环境更加美观舒适。无论您是在家中还是商业场所使用，我们的产品都能满足您的需求。购买我们的产品，您将获得一份合同使用资格的保证，让您放心使用。

选择我们的产品，让您的办公空间焕然一新！


## 2.1 问题一：生成文本太长

它似乎很好地完成了要求，即从技术说明书开始编写产品描述，介绍了一个精致的中世纪风格办公椅。但是当我看到这个时，我会觉得这个太长了。

所以在上述过程中，我产生想法后写了一个  Prompt ，并得到了结果，但是我对它不是很满意，因为它太长了。所以我澄清我的  Prompt ，要求它限制生成文本长度，要求最多使用50个字。


In [7]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our beautiful collection of medieval-style office furniture. This series includes file cabinets, desks, bookshelves, and conference tables. Choose from a variety of shell colors and base coatings. The chairs come with or without armrests and are suitable for both home and commercial use. Made in Italy.


提取回答并根据空格拆分，答案为54个字，较好地完成了设计要求。

In [8]:
lst = response.split()
print(len(lst))

47


In [9]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

使用最多50个词。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


我们的中世纪风格办公家具系列包括文件柜、办公桌、书柜和会议桌。可选择多种外壳颜色和底座涂层。椅子可选塑料或面料/皮革装饰，底座涂层可选不锈钢、黑色、白色或铬。座椅高度可调，适用于家庭或商业场所。尺寸为53x51x80厘米，座椅高度44厘米，座椅深度41厘米。可选择软地板或硬地板滚轮，两种座椅泡沫密度和有无扶手。外壳底座滑动件采用改性尼龙涂层的铸铝，座椅采用HD36泡沫。原产国意大利。


In [10]:
# 由于中文需要分词，此处直接计算整体长度
len(response)

193

LLM在能堪堪胜任严格的字数限制，但实现得并不精确。此例中，英文输出要求控制在50个词，但有时会输出60或65个单词的内容，但这也还算合理。原因是 LLM 使用分词器（tokenizer）解释文本，但它们往往在计算字符方面表现一般般。有很多不同的方法来尝试控制您得到的输出的长度（如若干句话/词/个汉字/个字母 (characters) 等）。

## 2.2 问题二：抓错文本细节

我们继续完善这段推广词，会发现的第二个问题是，这个网站并不是直接向消费者销售，它实际上面向的是家具零售商，他们会更关心椅子的技术细节和材料。在这种情况下，您可以继续修改这个  Prompt ，让它更精确地描述椅子的技术细节。

解决方法：要求它专注于与目标受众相关的方面。

In [13]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our beautiful mid-century style office furniture collection, featuring file cabinets, desks, bookshelves, and conference tables. Available in various shell colors and base coatings. Choose from plastic backrest decoration (SWC-100) or full decoration with 10 fabric and 6 leather options (SWC-110). Base coating options include stainless steel, matte black, glossy white, or chrome. Chairs available with or without armrests. Suitable for both residential and commercial spaces. Meets contract eligibility. Constructed with a plastic-coated aluminum base with five wheels for easy mobility. Pneumatic chair adjustment for convenient height control. Dimensions: Width 53cm | 20.87in, Depth 51cm | 20.08in, Height 80cm | 31.50in, Seat Height 44cm | 17.32in, Seat Depth 41cm | 16.14in. Choose between soft floor or hard floor casters. Two seat foam density options: medium (1.8 lbs/cubic ft) or high (2.8 lbs/cubic ft). Available with or without PU armrests in 8 positions. Made with modifie

In [14]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

使用最多50个单词。

技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

我们很高兴向您介绍我们的美丽中世纪风格办公家具系列。该系列包括文件柜、办公桌、书柜和会议桌等多种产品。我们提供多种外壳颜色和底座涂层供您选择。您可以选择塑料前后靠背装饰（SWC-100）或全面装饰（SWC-110），后者提供10种面料和6种皮革供您选择。底座涂层选项包括不锈钢、哑光黑色、光泽白色或铬。椅子可以选择带或不带扶手。我们的产品适用于家庭或商业场所，并符合合同使用资格。产品结构采用五个轮子的塑料涂层铝底座，并配备气动椅子调节功能，方便升降。产品尺寸为宽度53厘米、深度51厘米、高度80厘米、座椅高度44厘米和座椅深度41厘米。我们提供软地板或硬地板滚轮选项，以及两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。此外，您可以选择无扶手或8个位置PU扶手。我们的产品外壳底座滑动件采用改性尼龙PA6/PA66涂层的铸铝，外壳厚度为10毫米。座椅采用HD36泡沫制成。我们的产品原产国为意大利。


可见，通过修改  Prompt ，模型的关注点倾向了具体特征与技术细节。

我可能进一步想要在描述的结尾展示出产品ID。因此，我可以进一步改进这个  Prompt ，要求在描述的结尾，展示出说明书中的7位产品ID。

In [16]:
# 更进一步，要求在描述末尾包含 7个字符的产品ID
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our beautiful Mid-Century Style Office Furniture Collection, featuring file cabinets, desks, bookcases, and conference tables. Available in a variety of shell colors and base coatings. Choose from plastic backrest decoration (SWC-100) or full decoration with 10 fabric and 6 leather options (SWC-110). Base coating options include stainless steel, matte black, glossy white, or chrome. Chairs available with or without armrests. Suitable for both residential and commercial spaces. Meets contract eligibility.

Structure:
- Plastic-coated aluminum base with five wheels.
- Pneumatic chair adjustment for easy height control.

Dimensions:
- Width: 53cm | 20.87in
- Depth: 51cm | 20.08in
- Height: 80cm | 31.50in
- Seat Height: 44cm | 17.32in
- Seat Depth: 41cm | 16.14in

Options:
- Soft floor or hard floor wheel options.
- Two seat foam density options: medium (1.8 lbs/cubic ft) or high (2.8 lbs/cubic ft).
- Armrests available in 8 positions or without armrests.

Materials:
- Shell an

In [17]:
# 更进一步
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

使用最多50个单词。

技术规格： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

我们很高兴向您介绍我们的美丽中世纪风格办公家具系列。该系列包括文件柜、办公桌、书柜和会议桌等多种产品。我们提供多种外壳颜色和底座涂层供您选择。您可以选择塑料前后靠背装饰（SWC-100）或全面装饰（SWC-110），后者提供10种面料和6种皮革可供选择。底座涂层选项包括不锈钢、哑光黑色、光泽白色或铬。椅子可以选择带或不带扶手。我们的产品适用于家庭或商业场所，并符合合同使用资格。

产品的结构非常坚固，底座采用五个轮子的塑料涂层铝底座。椅子采用气动调节，方便升降。

产品的尺寸为宽度53厘米（20.87英寸），深度51厘米（20.08英寸），高度80厘米（31.50英寸），座椅高度44厘米（17.32英寸），座椅深度41厘米（16.14英寸）。

我们提供软地板或硬地板滚轮选项。座椅泡沫密度可选择中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。您还可以选择无扶手或8个位置PU扶手。

我们的产品采用高质量材料制成。外壳底座滑动件采用改性尼龙PA6/PA66涂层的铸铝，外壳厚度为10毫米。座椅采用HD36泡沫制成。

我们的产品原产国为意大利。

产品ID：SWC-100


以上是许多开发人员通常会经历的  Prompt 开发的迭代过程简短示例。我的建议是，像上一章中所演示的那样，Prompt 应该保持清晰和明确，并在必要时给模型一些思考时间。在这些要求的基础上，常见流程是首先尝试编写一版 Prompt ，看看会发生什么，然后继续迭代完善 Prompt，以逐渐接近所需的结果。许多成功的 Prompt 都是通过这种迭代过程得出的。我将向您展示一个更复杂的 Prompt 示例，可能会让您对 ChatGPT 的能力有更深入的了解。

## 2.3 问题三：添加表格描述
继续添加指引，要求提取产品尺寸信息并组织成表格，并指定表格的列、表名和格式；再将所有内容格式化为可以在网页使用的 HTML。

In [18]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>Product Description</h2>
  <p>
    Introducing our beautiful Mid-Century Style Office Furniture Collection, which includes file cabinets, desks, bookcases, and conference tables. This collection offers a variety of shell colors and base coatings to choose from. You can opt for plastic front and back seat decorations (SWC-100) or a comprehensive decoration with 10 fabric options and 6 leather options (SWC-110). The base coating options include stainless steel, matte black, glossy white, or chrome. The chairs are available with or without armrests. Suitable for both home and commercial spaces, these furniture pieces are contract-grade.
  </p>
  <p>
    The structure of these furniture pieces features a plastic-coated aluminum base with five wheels for easy mobility. The chairs are equipped with pneumatic height adjustment for convenient lifting and lowering.
  </p>
  <h3>Product Dimensions</h3>
  <table>
    <tr>
      <th>Width</th>
      <td>20.87 inches</td>
    </tr>
    

In [20]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

In [21]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

在描述之后，包括一个表格，提供产品的尺寸。表格应该有两列。第一列包括尺寸的名称。第二列只包括英寸的测量值。

给表格命名为“产品尺寸”。

将所有内容格式化为可用于网站的HTML格式。将描述放在<div>元素中。

技术规格：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
    <h2>产品描述</h2>
    <p>我们很自豪地介绍我们的美丽中世纪风格办公家具系列。这个系列包括文件柜、办公桌、书柜和会议桌等多种产品。我们提供多种外壳颜色和底座涂层供您选择。您可以选择塑料前后靠背装饰（SWC-100），也可以选择10种面料和6种皮革的全面装饰（SWC-110）。底座涂层选项包括不锈钢、哑光黑色、光泽白色或铬。椅子可以选择带扶手或不带扶手。我们的产品适用于家庭或商业场所，并符合合同使用资格。</p>
    <p>我们的产品采用高质量的材料和精心设计的结构。底座是由五个轮子的塑料涂层铝底座构成，方便移动和调整高度。椅子采用气动调节，可以轻松升降。</p>
    <h3>产品尺寸</h3>
    <table>
        <tr>
            <th>尺寸名称</th>
            <th>英寸测量值</th>
        </tr>
        <tr>
            <td>宽度</td>
            <td>20.87英寸</td>
        </tr>
        <tr>
            <td>深度</td>
            <td>20.08英寸</td>
        </tr>
        <tr>
            <td>高度</td>
            <td>31.50英寸</td>
        </tr>
        <tr>
            <td>座椅高度</td>
            <td>17.32英寸</td>
        </tr>
        <tr>
            <td>座椅深度</td>
            <td>16.14英寸</td>
        </tr>
    </table>
    <h3>选项</h3>
    <ul>
        <li>软地板或硬地板滚轮选项</li>
        <li>两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）</li>
        <li>无扶手或8个位置PU扶手</li>
    </ul>
    <h3>材料</h3>
    

In [22]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

本章的主要内容是 LLM 在开发应用程序中的迭代式  Prompt 开发过程。开发者需要先尝试编写  Prompt ，然后通过迭代逐步完善它，直至得到需要的结果。作为一名高效的提示词工程师（Prompt Engineer），关键在于掌握有效的开发Prompt的过程，而不是去寻求得到“完美的”Prompt。对于一些更复杂的应用程序，可以对多个样本（如数百张说明书）进行  Prompt 的迭代开发，并在样本集上进行评估。

最后，在更成熟的应用程序中，可以观察多个Prompt在多个样本集上的表现，测试平均或最差性能。但通常，**仅当**应用较成型之后，才推荐您通过这种评估方式，来精益求精。

请使用 Jupyter Notebook，动手实践本节给出的示例，并尝试不同的变化，查看结果。